In [2]:
import warnings
warnings.filterwarnings("ignore")
%load_ext autoreload
%autoreload 2
from qiskit_metal import Dict, MetalGUI, designs
from qiskit_metal.qlibrary.qubits.transmon_pocket import TransmonPocket
from qiskit_metal.qlibrary.qubits.transmon_pocket_6 import TransmonPocket6
from qiskit_metal.qlibrary.tlines.meandered import RouteMeander
from qiskit_metal.qlibrary.tlines.straight_path import RouteStraight
from qiskit_metal.qlibrary.terminations.launchpad_wb_coupled import LaunchpadWirebondCoupled
from qiskit_metal.qlibrary.couplers.coupled_line_tee import CoupledLineTee
from qiskit_metal.qlibrary.user_components.left_coupler import LeftCoupler
from qiskit_metal.qlibrary.user_components.right_coupler import RightCoupler
from qiskit_metal.qlibrary.user_components.straightdown_coupler import StraightDownCoupler
from qiskit_metal.qlibrary.user_components.straightup_coupler import StraightUpCoupler

In [3]:
design = designs.DesignFlipChip({}, overwrite_enabled=True)
gui = MetalGUI(design)

In [3]:
design.delete_all_components()

In [30]:
design.get_chip_z('Q_chip')

'5um'

In [4]:
design.chips['C_chip'].size['size_x']='8mm'
design.chips['C_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['size_x']='8mm'
design.chips['Q_chip'].size['size_y']='8mm'
design.chips['Q_chip'].size['center_z']='20um'

# Resonator length

In [5]:
from qiskit_metal.analyses.em.cpw_calculations import guided_wavelength

def find_resonator_length(frequency, line_width, line_gap, N): 
    #frequency in GHz
    #line_width/line_gap in um
    #N -> 2 for lambda/2, 4 for lambda/4
    
    [lambdaG, etfSqrt, q] = guided_wavelength(frequency*10**9, line_width*10**-6,
                                              line_gap*10**-6, 750*10**-6, 200*10**-9)
    return str(lambdaG/N*10**3)+" mm"

In [6]:
resonator_length_2=[]
for i in range(3):
    a=find_resonator_length(7.2+i*0.2,10,6,2)
    a=a.strip('mm')
    a=float(a)-0.8-0.2 
    resonator_length_2.append(a)
print(resonator_length_2)


[7.453254683870284, 7.224788339841421, 7.00834654020913]


# Flipchip

In [7]:
cp0 = dict(down = dict(loc_W=1, loc_H=-1, pad_width='120um'))

# q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
#                                             pos_y='-2000um',
#                                             chip='Q_chip',
#                                             connection_pads=cp0))]
q0=[TransmonPocket6(design, 'Q0', options=dict(pos_x='-2000um',
                                            pos_y='-2000um',
                                            chip='Q_chip'))]

qubits=q0

q0x=float(qubits[0].options.pos_x.strip('um'))
q0y=float(qubits[0].options.pos_y.strip('um'))


c0 = LeftCoupler(design, "c0", options=dict(pos_x=f"{q0x-60}um",
                                              pos_y=f"{q0y+80}um",
                                              pad_width="100um",
                                              pad_height="504um",
                                              cpw_stub_height="200um",
                                              orientation='270',                                            
                                              chip="C_chip"))

design.rebuild()
gui.rebuild()
gui.autoscale()

In [8]:
lpL=LaunchpadWirebondCoupled(design, 'lpL',
                                         options = dict(chip="C_chip",
                                         pos_x = '-3000um',
                                         pos_y = '2000um',
                                         lead_length = '30um'))
lpR=LaunchpadWirebondCoupled(design, 'lpR',
                                         options = dict(chip="C_chip",
                                         pos_x = '3000um',
                                         pos_y = '2000um',
                                         orientation='180',
                                         lead_length = '30um'))

In [9]:
clt0 = CoupledLineTee(design, "clt0", 
                        options=dict(pos_x='-2000um',
                                    pos_y='2000um', 
                                    coupling_length='300um',
                                    chip="C_chip",
                                    down_length='300um',
                                    fillet='99um',
                                    coupling_space='5um',
                                    mirror=True,
                                    open_termination=True))



In [10]:
fl0=RouteStraight(design, 'fl0',options=Dict(chip="C_chip", pin_inputs=Dict(start_pin=Dict(
                                                            component='lpL',
                                                            pin='tie'),
                                                            end_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_start'))))
fl1=RouteStraight(design, 'fl1',options=Dict(chip="C_chip",pin_inputs=Dict(start_pin=Dict(
                                                            component='clt0',
                                                            pin='prime_end'),
                                                            end_pin=Dict(
                                                            component='lpR',
                                                            pin='tie'))))


# Readout resonator

In [11]:
from collections import OrderedDict
jogs=OrderedDict()
def connect(component_name: str, component1: str, pin1: str, component2: str, pin2: str,
            length: str, start_jogged_extension: OrderedDict, end_jogged_extension: OrderedDict,
            spacing: str, asymmetry='0 um',  start_straight='0 um', end_straight='0 um', flip=False):
    """Connect two pins with a CPW."""
    myoptions = Dict(
        chip="C_chip",
        pin_inputs=Dict(
            start_pin=Dict(
                component=component1,
                pin=pin1),
            end_pin=Dict(
                component=component2,
                pin=pin2)),
        lead=Dict(
            start_straight=start_straight,
            end_straight=end_straight,
            start_jogged_extension=start_jogged_extension,
            end_jogged_extension=end_jogged_extension),
        total_length=length,
        fillet = '99.9um',
        meander=dict(asymmetry=asymmetry,
                     spacing=spacing))
    myoptions.meander.lead_direction_inverted = 'true' if flip else 'false'
    return RouteMeander(design, component_name, myoptions)

In [12]:
rr0=connect('rr0', 'clt0', 'second_end', 'c0', 'in',
            resonator_length_2[1],jogs,jogs, 
            '400um','0um', '100um', '100um')

# EPR Analysis

In [13]:
from qiskit_metal.analyses.quantization import EPRanalysis
eig_rd = EPRanalysis(design, "hfss")
hfss = eig_rd.sim.renderer

In [14]:
hfss.start()

INFO 10:39AM [connect_project]: Connecting to Ansys Desktop API...
INFO 10:39AM [load_ansys_project]: 	Opened Ansys App
INFO 10:39AM [load_ansys_project]: 	Opened Ansys Desktop v2018.0.0
INFO 10:39AM [load_ansys_project]: 	Opened Ansys Project
	Folder:    C:/Users/kimje/Documents/Ansoft/
	Project:   Project28
INFO 10:39AM [connect_design]: No active design found (or error getting active design).
INFO 10:39AM [connect]: 	 Connected to project "Project28". No design detected


True

In [15]:
hfss.activate_ansys_design("flip_V7", 'eigenmode')
hfss.render_design(['Q0','c0','lpL','lpR','clt0','rr0','fl0','fl1'],
                    [('clt0','prime_end')],
                    box_plus_buffer=False)

10:39AM 24s WARNING [activate_ansys_design]: The design_name=flip_V7 was not in active project.  Designs in active project are: 
[].  A new design will be added to the project.  
INFO 10:39AM [connect_design]: 	Opened active design
	Design:    flip_V7 [Solution type: Eigenmode]
WARNING 10:39AM [connect_setup]: 	No design setup detected.
WARNING 10:39AM [connect_setup]: 	Creating eigenmode default setup.
INFO 10:39AM [get_setup]: 	Opened setup `Setup`  (<class 'pyEPR.ansys.HfssEMSetup'>)


In [16]:
hfss.close()

True

# LOM Analysis

In [17]:
from qiskit_metal.analyses.quantization import LOManalysis
c1 = LOManalysis(design, "q3d")


In [18]:
c1.sim.setup.max_passes = 10
c1.sim.setup_update(freq_ghz='5.60',solution_order = 'Medium', auto_increase_solution_order = 'False')
c1.sim.setup

{'name': 'Setup',
 'reuse_selected_design': True,
 'reuse_setup': True,
 'freq_ghz': '5.60',
 'save_fields': False,
 'enabled': True,
 'max_passes': 10,
 'min_passes': 2,
 'min_converged_passes': 2,
 'percent_error': 0.5,
 'percent_refinement': 30,
 'auto_increase_solution_order': 'False',
 'solution_order': 'Medium',
 'solver_type': 'Iterative'}

In [20]:
#c1.sim.run(components=['Q0','c0'], open_terminations=[('c0', 'in')])
c1.sim._get_results_from_renderer()
c1.sim.capacitance_matrix

INFO 10:55AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpncix5fb7.txt, C, , Setup:LastAdaptive, "Original", "ohm", "nH", "fF", "mSie", 5600000000, Maxwell, 1, False
INFO 10:55AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpn6v_7y3o.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5600000000, Maxwell, 1, False
INFO 10:55AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpupjwduzj.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5600000000, Maxwell, 2, False
INFO 10:55AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpka3rum2b.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5600000000, Maxwell, 3, False
INFO 10:55AM [get_matrix]: Exporting matrix data to (C:\Users\kimje\AppData\Local\Temp\tmpq65s8gjx.txt, C, , Setup:AdaptivePass, "Original", "ohm", "nH", "fF", "mSie", 5600000000, Maxwell, 4, False
INFO 10:55

,ground,pad_bot_Q0,pad_top_Q0,quad_coupler_c0
ground,383.840,-74.71700,-55.506,-193.56000
pad_bot_Q0,-74.717,107.87000,-31.535,-0.56775
pad_top_Q0,-55.506,-31.53500,107.500,-19.37600
quad_coupler_c0,-193.560,-0.56775,-19.376,214.49000


In [21]:
c1.setup.junctions = Dict({'Lj': 11, 'Cj': 2})
c1.setup.freq_readout = 6.50
c1.setup.freq_bus = []
c1.run_lom()
c1.lumped_oscillator_all

[1, 2] [3]
Predicted Values

Transmon Properties
f_Q 5.404371 [GHz]
EC 272.526793 [MHz]
EJ 14.854145 [GHz]
alpha -310.728990 [MHz]
dispersion 4.983839 [KHz]
Lq 10.995564 [nH]
Cq 71.076417 [fF]
T1 22.626099 [us]

**Coupling Properties**

tCqbus1 -9.430870 [fF]
gbus1_in_MHz -125.528775 [MHz]
χ_bus1 -6.426297 [MHz]
1/T1bus1 7034.131055 [Hz]
T1bus1 22.626099 [us]
Bus-Bus Couplings


,fQ,EC,EJ,alpha,dispersion,gbus,chi_in_MHz,χr MHz,gr MHz
1,5.850756,322.624099,14.854145,-373.332508,25.47916,[-19.960907535860024],[-0.45011876407289714],0.450119,19.960908
2,5.642749,298.682666,14.854145,-343.212278,12.307332,[-196.0668080231643],[-25.825726670501684],25.825727,196.066808
3,5.567809,290.31358,14.854145,-332.770771,9.341625,[-175.59172663146356],[-17.546983036058908],17.546983,175.591727
4,5.575149,291.127309,14.854145,-333.784022,9.600519,[-158.68253561869204],[-14.559063350608906],14.559063,158.682536
5,5.52696,285.808378,14.854145,-327.168622,8.012395,[-142.51485087925502],[-10.599018895466108],10.599019,142.514851
6,5.478929,280.562043,14.854145,-320.661315,6.66945,[-131.49693772577822],[-8.173836339231782],8.173836,131.496938
7,5.448316,277.246878,14.854145,-316.558446,5.923319,[-129.14747919570033],[-7.4144870464411134],7.414487,129.147479
8,5.433143,275.611993,14.854145,-314.537695,5.582242,[-126.57409183826779],[-6.911312510004948],6.911313,126.574092
9,5.415104,273.675383,14.854145,-312.146223,5.199913,[-126.31557031783524],[-6.644152307509199],6.644152,126.315570
10,5.404371,272.526793,14.854145,-310.72899,4.983839,[-125.52877481689835],[-6.426297383446051],6.426297,125.528775


In [22]:
c1.sim.close()

Warning! 3 COM references still alive
Ansys will likely refuse to shut down
